In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-03-23 15:13:46.850529: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 15:13:46.874931: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 15:13:46.874955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 15:13:46.874975: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 15:13:46.879518: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-23 15:13:49.496310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 15:13:49.525065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 15:13:49.525281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))
bird_classes

['BAY-BREASTED WARBLER',
 'BLUE GROSBEAK',
 'CAPE MAY WARBLER',
 'CERULEAN WARBLER',
 'HEPATIC TANAGER',
 'INDIGO BUNTING',
 'LAZULI BUNTING',
 'OVENBIRD',
 'PYRRHULOXIA',
 'ROSE BREASTED GROSBEAK',
 'SCARLET TANAGER',
 'TOWNSENDS WARBLER',
 'acorn_woodpecker',
 'american_bittern',
 'american_dipper',
 'american_flamingo',
 'american_pipit',
 'american_robin',
 'american_three_toed_woodpecker',
 'american_wigeon',
 'amethyst_throated_mountain_gem',
 'anhinga',
 "anna's_hummingbird",
 'baikal_teal',
 'baltimore_oriole',
 'barn_swallow',
 'barrows_goldeneye',
 'bay_breasted_warbler',
 'black_and_white_warbler',
 'black_bellied_whistling_duck',
 'black_capped_chickadee',
 'black_chinned_hummingbird',
 'black_headed_grosbeak',
 'black_throated_sparrow',
 'black_vented_shearwater',
 'black_vulture',
 'blue_gray_gnatcatcher',
 'blue_grosbeak',
 'blue_jay',
 'blue_winged_warbler',
 'bobolink',
 'boreal_owl',
 'brant_goose',
 'brewers_blackbird',
 'broad_billed_hummingbird',
 'broad_tailed_hum

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 25744 files belonging to 167 classes.
Using 18021 files for training.


2024-03-23 15:13:51.474499: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 15:13:51.474751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 15:13:51.474900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 25744 files belonging to 167 classes.
Using 7723 files for validation.
Found 11545 files belonging to 167 classes.


In [4]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
base_model = tf.keras.applications.EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False

for layer in base_model.layers:
    print(layer.name)

258076736/258076736 [==============================] - 50s 0us/step
input_1
rescaling
normalization
rescaling_1
stem_conv_pad
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block1b_dwconv
block1b_bn
block1b_activation
block1b_se_squeeze
block1b_se_reshape
block1b_se_reduce
block1b_se_expand
block1b_se_excite
block1b_project_conv
block1b_project_bn
block1b_drop
block1b_add
block1c_dwconv
block1c_bn
block1c_activation
block1c_se_squeeze
block1c_se_reshape
block1c_se_reduce
block1c_se_expand
block1c_se_excite
block1c_project_conv
block1c_project_bn
block1c_drop
block1c_add
block1d_dwconv
block1d_bn
block1d_activation
block1d_se_squeeze
block1d_se_reshape
block1d_se_reduce
block1d_se_expand
block1d_se_excite
block1d_project_conv
block1d_project_bn
block1d_drop
block1d_add
block2a_expand_conv
block2a_expand_bn
block2a_expand_acti

In [6]:
inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = layers.Dense(len(bird_classes), activation='softmax', dtype=tf.float32)(x)

model_1_eff_net_b0 = tf.keras.Model(inputs, outputs)

In [7]:
model_1_eff_net_b0.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 2560)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1311232   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328

In [8]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3) 

In [9]:
model_1_eff_net_b0.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='Best_models/best_eff_b7/', verbose=1, save_best_only=True)

history_eff_b0 = model_1_eff_net_b0.fit(train_dataset,
                                        steps_per_epoch=len(train_dataset),
                                        validation_data=validation_dataset,
                                        validation_steps=len(validation_dataset),
                                        epochs=10,
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 1/10


2024-03-23 15:16:02.637777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-23 15:16:02.696299: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-23 15:16:03.151965: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 761.60MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-23 15:16:03.151997: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 761.60MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  1/564 [..............................] - ETA: 1:34:48 - loss: 5.1888 - accuracy: 0.0000e+00

2024-03-23 15:16:03.958304: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7c4344003530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-23 15:16:03.958324: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-23 15:16:03.965457: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-23 15:16:04.036084: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


564/564 [==============================] - ETA: 0s - loss: 3.1398 - accuracy: 0.2565
Epoch 1: val_loss improved from inf to 1.17008, saving model to Best_models/best_eff_b7/
564/564 [==============================] - 154s 256ms/step - loss: 3.1398 - accuracy: 0.2565 - val_loss: 1.1701 - val_accuracy: 0.6749 - lr: 0.0010
Epoch 2/10
563/564 [============================>.] - ETA: 0s - loss: 1.7239 - accuracy: 0.4965
Epoch 2: val_loss improved from 1.17008 to 0.74235, saving model to Best_models/best_eff_b7/
564/564 [==============================] - 144s 255ms/step - loss: 1.7243 - accuracy: 0.4964 - val_loss: 0.7423 - val_accuracy: 0.7458 - lr: 0.0010
Epoch 3/10
563/564 [============================>.] - ETA: 0s - loss: 1.3314 - accuracy: 0.5894
Epoch 3: val_loss improved from 0.74235 to 0.56672, saving model to Best_models/best_eff_b7/
564/564 [==============================] - 141s 249ms/step - loss: 1.3314 - accuracy: 0.5893 - val_loss: 0.5667 - val_accuracy: 0.7782 - lr: 0.0010
Epoc

In [13]:
print(len(base_model.layers))
for layer in base_model.layers[-150:]:
    layer.trainable = True


814


In [14]:
model_1_eff_net_b0.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 10

model_1_eff_net_b0.fit(train_dataset,
                                        steps_per_epoch=len(train_dataset),
                                        validation_data=validation_dataset,
                                        validation_steps=len(validation_dataset),
                                        epochs=start_epoch+10,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 11/20
563/564 [============================>.] - ETA: 0s - loss: 0.6275 - accuracy: 0.7831
Epoch 11: val_loss improved from 0.32924 to 0.30918, saving model to Best_models/best_eff_b7/
564/564 [==============================] - 159s 267ms/step - loss: 0.6274 - accuracy: 0.7832 - val_loss: 0.3092 - val_accuracy: 0.8800
Epoch 12/20
563/564 [============================>.] - ETA: 0s - loss: 0.5968 - accuracy: 0.7932
Epoch 12: val_loss did not improve from 0.30918
564/564 [==============================] - 107s 190ms/step - loss: 0.5967 - accuracy: 0.7932 - val_loss: 0.3299 - val_accuracy: 0.8457
Epoch 13/20
563/564 [============================>.] - ETA: 0s - loss: 0.5717 - accuracy: 0.8007
Epoch 13: val_loss improved from 0.30918 to 0.28561, saving model to Best_models/best_eff_b7/
564/564 [==============================] - 150s 266ms/step - loss: 0.5717 - accuracy: 0.8007 - val_loss: 0.2856 - val_accuracy: 0.8881
Epoch 14/20
563/564 [============================>.] - ETA: 0s - los

In [30]:

for layer in base_model.layers[-800:]:
    layer.trainable = True


In [34]:
model_1_eff_net_b0.compile(loss='categorical_crossentropy',
                           optimizer=Adam(0.00003125),
                           metrics=['accuracy'])
start_epoch = 68

model_1_eff_net_b0.fit(train_dataset,
                                        steps_per_epoch=len(train_dataset),
                                        validation_data=validation_dataset,
                                        validation_steps=len(validation_dataset),
                                        epochs=start_epoch+5,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 69/73
563/564 [============================>.] - ETA: 0s - loss: 0.1463 - accuracy: 0.9253
Epoch 69: val_loss did not improve from 0.18998
564/564 [==============================] - 121s 195ms/step - loss: 0.1463 - accuracy: 0.9253 - val_loss: 0.1999 - val_accuracy: 0.9451 - lr: 3.1250e-05
Epoch 70/73
563/564 [============================>.] - ETA: 0s - loss: 0.1400 - accuracy: 0.9267
Epoch 70: val_loss did not improve from 0.18998
564/564 [==============================] - 108s 191ms/step - loss: 0.1401 - accuracy: 0.9266 - val_loss: 0.1984 - val_accuracy: 0.9538 - lr: 3.1250e-05
Epoch 71/73
563/564 [============================>.] - ETA: 0s - loss: 0.1447 - accuracy: 0.9268
Epoch 71: val_loss did not improve from 0.18998
564/564 [==============================] - 108s 191ms/step - loss: 0.1447 - accuracy: 0.9268 - val_loss: 0.1985 - val_accuracy: 0.9548 - lr: 3.1250e-05
Epoch 72/73
563/564 [============================>.] - ETA: 0s - loss: 0.1430 - accuracy: 0.9252
Epoch 72: va

In [35]:
model_1_eff_net_b0.evaluate(test_dataset)

360/361 [============================>.] - ETA: 0s - loss: 0.3100 - accuracy: 0.9085

2024-03-23 17:58:23.480416: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 598.51MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-23 17:58:23.481766: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 598.51MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


361/361 [==============================] - 50s 137ms/step - loss: 0.3093 - accuracy: 0.9087


[0.30928918719291687, 0.9087050557136536]